In [1]:
def cross(string1, string2):
    return [s + r for s in string1 for r in string2]

rows = "ABCDEDGHI"
cols = "123456789"

boxes = cross(rows, cols)

In [2]:
row_units = [cross(r, cols) for r in rows]
col_units = [cross(rows, c) for c in cols]


In [3]:
square_units = [cross(rs, cs) for rs in ["ABC","DEF","GHI"] for cs in ["123", "456", "789"]]

In [4]:
unit_list = row_units + col_units + square_units

In [7]:
for unit in unit_list:
    print(unit)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9']
['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9']
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9']
['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9']
['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9']
['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']
['A1', 'B1', 'C1', 'D1', 'E1', 'D1', 'G1', 'H1', 'I1']
['A2', 'B2', 'C2', 'D2', 'E2', 'D2', 'G2', 'H2', 'I2']
['A3', 'B3', 'C3', 'D3', 'E3', 'D3', 'G3', 'H3', 'I3']
['A4', 'B4', 'C4', 'D4', 'E4', 'D4', 'G4', 'H4', 'I4']
['A5', 'B5', 'C5', 'D5', 'E5', 'D5', 'G5', 'H5', 'I5']
['A6', 'B6', 'C6', 'D6', 'E6', 'D6', 'G6', 'H6', 'I6']
['A7', 'B7', 'C7', 'D7', 'E7', 'D7', 'G7', 'H7', 'I7']
['A8', 'B8', 'C8', 'D8', 'E8', 'D8', 'G8', 'H8', 'I8']
['A9', 'B9', 'C9', 'D9', 'E9', 'D9', 'G9', 'H9', 'I9']
['A1', 'A2

In [10]:
units = dict((s, [u for u in unit_list if s in u]) for s in boxes)


In [25]:
peers = dict((s, set(sum(units[s], [])) - set([s])) for s in boxes)

In [27]:
def display(values):
    """
    display the values in 2D Format
    :param values: Sudoku in dictionary form
    :return: None
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)

    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [29]:
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    values = []
    digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(digits)
        elif c in digits:
            values.append(c)
    assert len(values)==81 
    return dict(zip(boxes, values))
    

In [51]:
def eliminat(values):
    """
    Eliminate values from peers of each box which has single values
    :param values: sudoku in a dictionary form
    :return: sudoko in dictioonary form after elimination
    """
    solved_boxs = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_boxs:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit, '')
    return values
            

In [52]:
def only_choice(values):
    """
    go through all the units, and whenever there is a unit with a value that only fits in the box, assign the value to 
    this box
    :param values: sudoku in a dictionary form
    :return: resulting sudoku in dictionary form after filling in only choices.
    """
    for unit in unit_list:
        for digit in "123456789":
            dplace = [box for box in unit if digit in values[box]]
            if len(dplace) == 1:
                values[dplace[0]] = digit
    return values    


In [58]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). if at some point, there is a box with no available value, if the sudoku is solved
    return sudoku, if after iteration sudoku remains the same, return sudoku
    :param values: a sudoku in dictionary form
    :return: the resulting sudoku in dictionary format
    """
    
    stoped = False
    while not stoped:
        solved_boxes_before = len([box for box in values.keys() if len(values[box])== 1])
        values = eliminat(values)
        values = only_choice(values)
        solved_boxes_after = len([box for box in values.keys() if len(values[box])== 1])
        stoped = solved_boxes_after == solved_boxes_before
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [54]:
def search(values):
    """
    using depth-first search and propagation, try all possible values.
    :param values: a sudoku in dictionary form
    :return: the resulting sudoku in dictionary format
    """
    values = reduce_puzzle(values)
    if values is False:
        return False # didnt solve!
    if all(len(values[s]) for s in boxes):
        return values ## Solved!
    # choose one of the unfilled squares with the fewest possibilities
    n, s = min(len(values[s], s) for s in boxes if len(values[s]) > 1)
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attemp = search(new_sudoku)
        if attemp:
            return attemp 
    

In [ ]:
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'


In [56]:
values = grid_values(diag_sudoku_grid)

In [60]:
eliminat(values)

KeyError: 'F4'

In [59]:
display(values)

    2      3456789   3456789  | 1345789   1345789   1345789  | 1345689   1345689   1345689  
  345789    345789    345789  | 1345789   1345789      6     |    2       134589    134589  
  345689    345689      1     |  234589    234589    234589  |  345689      7       345689  
------------------------------+------------------------------+------------------------------
 13456789 123456789  23456789 |    6     123456789  12345789 |    4      12345689 123456789 
    3     123456789  23456789 |123456789     9      12345789 | 13456789  12345689     7     
 13456789 123456789  23456789 |    6     123456789  12345789 |    4      12345689 123456789 
 13456789     4      23456789 |123456789 123456789  12345789 |    8      12345689 123456789 
 13456789 123456789     5     |    2     123456789  12345789 | 13456789  12345689 123456789 
 13456789 123456789  23456789 | 12345789 123456789  12345789 | 13456789  12345689     3     
